# Парсер сайта наш-дом-рф (https://наш.дом.рф/)
## Функционал :
#### 1. Парсинг сайта с формированием объекта pandas.DataFrame с информацией по объектам строительства.
#### 2. Сохранение полученного pandas.DataFrame в 3 файла с расширениями .sqlite .xlsx .pkl (директория ./data/).
#### 3. Скачивание фотографий (директория ./data/photos/) всех этапов строительства по указанному id объекта (id объекта можно взять из сформированного pandas.DataFrame, либо из созданных файлов).

#### Импорты :

In [74]:
import requests
import json
import os
import pandas as pd
import sqlite3
import time
from tqdm.notebook import tqdm

In [75]:
def send_request(url):
    response = requests.get(url)
    if response.status_code != 200:
        response.raise_for_status()
    return response

#### Формирование Датафрейма :

In [81]:
def get_objects_ids(n_obj, send_req=send_request):
    print("Формирование списка id объектов строительства")
    def get_pagination_api_url(_url_offset_param, _url_limit_param):
        return "https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object" \
               f"?offset={_url_offset_param}&" \
               f"limit={_url_limit_param}&" \
               "sortField=devId.devShortCleanNm&sortType=asc&objStatus=0"
    objs_ids = []
    url_offset_param = 0
    url_limit_param = n_obj if n_obj != -1 else 1000
    condition = True
    while condition:
        url = get_pagination_api_url(url_offset_param, url_limit_param)
        response = send_req(url)
        response_json = response.json()
        response_objects = response_json["data"]["list"]
        n_objects_in_response = len(response_objects)
        for response_obj in response_objects:
            obj_id = str(response_obj["objId"])
            objs_ids.append(obj_id)
        url_offset_param += url_limit_param
        n_objects_collected = len(objs_ids)
        if n_objects_in_response == 0:
            condition = False
        if n_objects_collected >= n_obj and n_obj != -1:
            condition = False
    print("Список id успешно создан\n---\n")
    return objs_ids


def collect_objects_data(obj_ids, send_req=send_request):
    print(f"Формирование словаря с данными по каждому из {len(obj_ids)} объектов")
    def get_obj_api_url(_obj_id):
        return f"https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/{_obj_id}"
    objs_data_dict = {}
    for obj_id in tqdm(obj_ids):
        url = get_obj_api_url(obj_id)
        response = send_req(url)
        json_response = response.json()
        response_data = json_response["data"]
        objs_data_dict[obj_id] = response_data
    print(f"Словарь данных по {len(obj_ids)} объектам успешно создан\n---\n")
    return objs_data_dict


def create_objs_df(objs_data_dict):
    print(f"Формирование Датафрейма по {len(objs_data_dict)} объектам из полученного словаря")
    objs_df = pd.DataFrame()
    def get_one_layer_dict(dict_before_resize):
        one_layer_dict = {}
        def dict_resize(key, dict_value):
            if type(dict_value) == dict:
                for key in dict_value:
                    dict_resize(key, dict_value[key])
            else:
                one_layer_dict[key] = dict_value
        for dict_key in dict_before_resize:
            dict_resize(dict_key, dict_before_resize[dict_key])
        return one_layer_dict
    for obj_id in objs_data_dict:
        obj_data_dict = objs_data_dict[obj_id]
        obj_data_dict = get_one_layer_dict(obj_data_dict)
        obj_data_series = pd.Series(obj_data_dict, name=obj_id)
        objs_df = objs_df.append(obj_data_series, ignore_index=False)
    print("Датафрейм успешно создан")
    return objs_df

In [82]:
# Формирование датафрейма
n_obj = 40 # колличество объектов по которым будут запросы на сайт, если нужны все объекты, то ввести -1
objects_ids_list = get_objects_ids(n_obj=n_obj)
objects_data_dict = collect_objects_data(objects_ids_list)
df = create_objs_df(objects_data_dict)

Формирование списка id объектов строительства
Список id успешно создан
---

Формирование словаря с данными по каждому из 40 объектов



Словарь данных по 40 объектам успешно создан
---

Формирование Датафрейма по 40 объектам из полученного словаря
Датафрейм успешно создан


In [83]:
df.head()

,address,buildObjCnt,comissObjCnt,conclusion,devEmail,devEmplMainFullNm,devFactAddr,devFullCleanNm,devId,devInn,...,companyGroupId,developerGroupName,color,colors,isWalk,line,name,time,transportDistIndex,complexShortNm
25002,"село Озерецкое, д. 30",1.0,18.0,0.0,resnov@cmft.ru,Меламуд Александр Романович,обл Московская Дмитровский город Дмитров село ...,10 КВАРТАЛ,2666.0,5050102110,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13391,"г Тюмень, ул Профсоюзная",5.0,0.0,0.0,info@2mengroup.ru,Киселев Сергей Михайлович,"Тюменская область, город Тюмень, улица Комсомо...",2МЕН ГРУПП,306.0,7701651356,...,699421001.0,2МЕН ГРУПП ДЕВЕЛОПМЕНТ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13383,г Тюмень,5.0,0.0,0.0,info@2mengroup.ru,Киселев Сергей Михайлович,"Тюменская область, город Тюмень, улица Комсомо...",2МЕН ГРУПП,306.0,7701651356,...,699421001.0,2МЕН ГРУПП ДЕВЕЛОПМЕНТ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13397,"г Тюмень, ул Профсоюзная, д. 56",5.0,0.0,0.0,info@2mengroup.ru,Киселев Сергей Михайлович,"Тюменская область, город Тюмень, улица Комсомо...",2МЕН ГРУПП,306.0,7701651356,...,699421001.0,2МЕН ГРУПП ДЕВЕЛОПМЕНТ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13398,"г Тюмень, ул Профсоюзная, д. 56",5.0,0.0,0.0,info@2mengroup.ru,Киселев Сергей Михайлович,"Тюменская область, город Тюмень, улица Комсомо...",2МЕН ГРУПП,306.0,7701651356,...,699421001.0,2МЕН ГРУПП ДЕВЕЛОПМЕНТ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Запись файлов :

In [84]:
def save_df_as_sqlite(df, fname="objects_data"):
    print(f"Запись Датафрейма в файл {fname}.sqlite")
    df = df.astype(str)
    if "data" not in os.listdir():
        os.mkdir(f"./data")
    conn = sqlite3.connect(f"./data/{fname}.sqlite")
    df.to_sql(fname, conn, if_exists="replace", index=False)
    print(f"Файл {fname}.sqlite успешно записан\n---\n")
    
    
def save_df_as_xlsx(df, fname="objects_data"):
    print(f"Запись Датафрейма в файл {fname}.xlsx")
    df.to_excel(f"./data/{fname}.xlsx", engine="openpyxl")
    print(f"Файл {fname}.sqlite успешно записан\n---\n")

    
def save_df_as_pickle(df, fname="objects_data"):
    print(f"Запись Датафрейма в файл {fname}.pkl")
    df.to_pickle(f"./data/{fname}.pkl")
    print(f"Файл {fname}.sqlite успешно записан")

In [85]:
# Запись df в файлы
save_df_as_sqlite(df)
save_df_as_xlsx(df)
save_df_as_pickle(df)

Запись Датафрейма в файл objects_data.sqlite
Файл objects_data.sqlite успешно записан
---

Запись Датафрейма в файл objects_data.xlsx
Файл objects_data.sqlite успешно записан
---

Запись Датафрейма в файл objects_data.pkl
Файл objects_data.sqlite успешно записан


#### Чтение файлов :

In [86]:
def read_df_from_sqlite(fname="objects_data"):
    print(f"Формирование Датафрейма из файла {fname}.sqlite")
    conn = sqlite3.connect(f"./data/{fname}.sqlite")
    df_sql = pd.read_sql(f"select * from {fname}", conn)
    print("Датафрейм успешно создан\n---\n")
    return df_sql


def read_df_from_xlsx(fname="objects_data"):
    print(f"Формирование Датафрейма из файла {fname}.sqlite")
    df_xlsx = pd.read_excel(f"./data/{fname}.xlsx")
    print("Датафрейм успешно создан\n---\n")
    return df_xlsx


def read_df_from_pickle(fname="objects_data"):
    print(f"Формирование Датафрейма из файла {fname}.pkl")
    df_xlsx = pd.read_pickle(f"./data/{fname}.pkl")
    print("Датафрейм успешно создан")
    return df_xlsx

In [87]:
# Создание датафрейма по считанному файлу (типы данных всех значений в данных датафреймах будут object, т.к. при записи 
# значения ячеек конвертровались в строки)
df_from_sqlite = read_df_from_sqlite()
df_from_xlsx = read_df_from_xlsx()
df_from_pkl = read_df_from_pickle()

Формирование Датафрейма из файла objects_data.sqlite
Датафрейм успешно создан
---

Формирование Датафрейма из файла objects_data.sqlite
Датафрейм успешно создан
---

Формирование Датафрейма из файла objects_data.pkl
Датафрейм успешно создан


In [88]:
df_from_sqlite.head()

,address,buildObjCnt,comissObjCnt,conclusion,devEmail,devEmplMainFullNm,devFactAddr,devFullCleanNm,devId,devInn,...,companyGroupId,developerGroupName,color,colors,isWalk,line,name,time,transportDistIndex,complexShortNm
0,"село Озерецкое, д. 30",1.0,18.0,0.0,resnov@cmft.ru,Меламуд Александр Романович,обл Московская Дмитровский город Дмитров село ...,10 КВАРТАЛ,2666.0,5050102110,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,"г Тюмень, ул Профсоюзная",5.0,0.0,0.0,info@2mengroup.ru,Киселев Сергей Михайлович,"Тюменская область, город Тюмень, улица Комсомо...",2МЕН ГРУПП,306.0,7701651356,...,699421001.0,2МЕН ГРУПП ДЕВЕЛОПМЕНТ,nan,nan,nan,nan,nan,nan,nan,nan
2,г Тюмень,5.0,0.0,0.0,info@2mengroup.ru,Киселев Сергей Михайлович,"Тюменская область, город Тюмень, улица Комсомо...",2МЕН ГРУПП,306.0,7701651356,...,699421001.0,2МЕН ГРУПП ДЕВЕЛОПМЕНТ,nan,nan,nan,nan,nan,nan,nan,nan
3,"г Тюмень, ул Профсоюзная, д. 56",5.0,0.0,0.0,info@2mengroup.ru,Киселев Сергей Михайлович,"Тюменская область, город Тюмень, улица Комсомо...",2МЕН ГРУПП,306.0,7701651356,...,699421001.0,2МЕН ГРУПП ДЕВЕЛОПМЕНТ,nan,nan,nan,nan,nan,nan,nan,nan
4,"г Тюмень, ул Профсоюзная, д. 56",5.0,0.0,0.0,info@2mengroup.ru,Киселев Сергей Михайлович,"Тюменская область, город Тюмень, улица Комсомо...",2МЕН ГРУПП,306.0,7701651356,...,699421001.0,2МЕН ГРУПП ДЕВЕЛОПМЕНТ,nan,nan,nan,nan,nan,nan,nan,nan


In [89]:
df_from_xlsx.head()

,Unnamed: 0,address,buildObjCnt,comissObjCnt,conclusion,devEmail,devEmplMainFullNm,devFactAddr,devFullCleanNm,devId,...,companyGroupId,developerGroupName,color,colors,isWalk,line,name,time,transportDistIndex,complexShortNm
0,25002,"село Озерецкое, д. 30",1,18,0,resnov@cmft.ru,Меламуд Александр Романович,обл Московская Дмитровский город Дмитров село ...,10 КВАРТАЛ,2666,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,13391,"г Тюмень, ул Профсоюзная",5,0,0,info@2mengroup.ru,Киселев Сергей Михайлович,"Тюменская область, город Тюмень, улица Комсомо...",2МЕН ГРУПП,306,...,699421001.0,2МЕН ГРУПП ДЕВЕЛОПМЕНТ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,13383,г Тюмень,5,0,0,info@2mengroup.ru,Киселев Сергей Михайлович,"Тюменская область, город Тюмень, улица Комсомо...",2МЕН ГРУПП,306,...,699421001.0,2МЕН ГРУПП ДЕВЕЛОПМЕНТ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,13397,"г Тюмень, ул Профсоюзная, д. 56",5,0,0,info@2mengroup.ru,Киселев Сергей Михайлович,"Тюменская область, город Тюмень, улица Комсомо...",2МЕН ГРУПП,306,...,699421001.0,2МЕН ГРУПП ДЕВЕЛОПМЕНТ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,13398,"г Тюмень, ул Профсоюзная, д. 56",5,0,0,info@2mengroup.ru,Киселев Сергей Михайлович,"Тюменская область, город Тюмень, улица Комсомо...",2МЕН ГРУПП,306,...,699421001.0,2МЕН ГРУПП ДЕВЕЛОПМЕНТ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [90]:
df_from_pkl.head()

,address,buildObjCnt,comissObjCnt,conclusion,devEmail,devEmplMainFullNm,devFactAddr,devFullCleanNm,devId,devInn,...,companyGroupId,developerGroupName,color,colors,isWalk,line,name,time,transportDistIndex,complexShortNm
25002,"село Озерецкое, д. 30",1.0,18.0,0.0,resnov@cmft.ru,Меламуд Александр Романович,обл Московская Дмитровский город Дмитров село ...,10 КВАРТАЛ,2666.0,5050102110,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13391,"г Тюмень, ул Профсоюзная",5.0,0.0,0.0,info@2mengroup.ru,Киселев Сергей Михайлович,"Тюменская область, город Тюмень, улица Комсомо...",2МЕН ГРУПП,306.0,7701651356,...,699421001.0,2МЕН ГРУПП ДЕВЕЛОПМЕНТ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13383,г Тюмень,5.0,0.0,0.0,info@2mengroup.ru,Киселев Сергей Михайлович,"Тюменская область, город Тюмень, улица Комсомо...",2МЕН ГРУПП,306.0,7701651356,...,699421001.0,2МЕН ГРУПП ДЕВЕЛОПМЕНТ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13397,"г Тюмень, ул Профсоюзная, д. 56",5.0,0.0,0.0,info@2mengroup.ru,Киселев Сергей Михайлович,"Тюменская область, город Тюмень, улица Комсомо...",2МЕН ГРУПП,306.0,7701651356,...,699421001.0,2МЕН ГРУПП ДЕВЕЛОПМЕНТ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13398,"г Тюмень, ул Профсоюзная, д. 56",5.0,0.0,0.0,info@2mengroup.ru,Киселев Сергей Михайлович,"Тюменская область, город Тюмень, улица Комсомо...",2МЕН ГРУПП,306.0,7701651356,...,699421001.0,2МЕН ГРУПП ДЕВЕЛОПМЕНТ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Скачивание фотографий :

In [91]:
def download_photos(obj_id, send_req=send_request):
    print(f"Скачивание фотографий этапов строительства объекта с id - {obj_id}")
    def get_obj_url(_obj_id):
        return f"https://наш.дом.рф/сервисы/каталог-новостроек/объект/{_obj_id}"
    url = get_obj_url(obj_id)
    response = send_req(url)
    obj_html_page = response.text
    obj_html_page_photos_string = obj_html_page[obj_html_page.index("\"shortInfo\":") + \
                                                len("\"shortInfo\":"): \
                                                obj_html_page.index(",\"streams\"")]
    obj_photos_json = json.loads(obj_html_page_photos_string)
    counter_for_fname = 0
    if "photos" not in os.listdir("./data"):
        os.mkdir(f"./data/photos")
    for obj_photo_json in tqdm(obj_photos_json):
        obj_photo_json_file_name = obj_photo_json["filename"].split(".")[0]
        obj_photo_json_file_ext = obj_photo_json["filename"].split(".")[1]
        fname = f"{obj_photo_json_file_name}_{counter_for_fname}.{obj_photo_json_file_ext}"
        if str(obj_id) not in os.listdir("./data/photos"):
            os.mkdir(f"./data/photos/{obj_id}")
        url = obj_photo_json["src"]
        response = send_req(url)
        with open(f"./data/photos/{obj_id}/{fname}", "wb") as photo:
            photo.write(response.content)
        counter_for_fname += 1
    print(f"Фотографии ({len(obj_photos_json)} успешно скачаны\n---\n")

In [ ]:
# Скачивание фотографий по указанному id объекта.
object_id = 20524
download_photos(object_id)

Скачивание фотографий этапов строительства объекта с id - 20524
